<h1 style="text-align: center;">Desafío - Gráficos y correlación</h1>

- Para realizar este desafío debes haber estudiado previamente todo el material disponible correspondiente a la unidad.
- Una vez terminado el desafío, comprime la carpeta que contiene el desarrollo de los requerimientos solicitados y sube el .zip en el LMS.
- Desarrollo del desafío: individual
- Para la realización del desafío necesitarás apoyarte del archivo `Apoyo Desafío - Gráficos y correlación`, disponible en la <a href="https://empieza.desafiolatam.com/cohorts/2108/sections/79636/activities/120921/items/617749">plataforma</a>.

## Habilidades a evaluar
- Reconocer las funcionalidades avanzadas de gráficos estáticos mediante `seaborn`.
- Aprender a realizar gráficos que muestren de forma estratificada el comportamiento de subconjuntos de elementos en la muestra.
- Entender e interpretar la correlación a partir de diagramas de dispersión.

## Descripción
La empresa para la cual trabajas, te ha solicitado continuar el análisis de datos pero, esta vez, a partir de gráficos existentes y la correlación que existe entre ellos con la data disponible. Para ello, te entregan un material de apoyo el cual deberá servir como base para desarrollar los siguientes requerimientos.

## Requerimientos
A continuación revisaremos los requerimientos y acciones que te han sido solicitados:

### 1. Importar librerías y .csv. (2 punto) para ello:
- Importe las librerías básicas para el análisis de datos
- Descargue e importe el archivo nations.csv.

**Tip**: El formato del archivo posee una codificación de tipo `iso-8859-1`. Investigue cómo se puede solucionar este problema utilizando el argumento encoding de `pd.read_csv`.

La base de datos contiene información a nivel mundial sobre demografía:
- `country`: País.
- `region`: Continente del país.
- `gdp`: Producto Interno Bruto per cápita, precios 2005.
- `school`: Promedio años de escolaridad.
- `adfert`: Fertilidad adolescente (Nacimientos 1:1000 en mujeres entre 15 y 19).
- `chldmort`: Probabilidad de muerte antes de los 5 años por cada 1000.
- `life`: Esperanza de vida al nacer.
- `pop`: Población total.
- `urban`: Porcentaje de población urbana.
- `femlab`: Tasa entre hombres y mujeres en el mercado laboral.
- `literacy`: Tasa de alfabetismo.
- `co2`: Toneladas de Co2 mitidas per cápita.
- `gini`: Coeficiente de desigualdad del ingreso.
- **Apellidos desde la A hasta la N**: Enfocarse en las variables `chldmort`, `adfert` y `life`.
- **Apellidos desde la M hasta la Z**: Enfocarse en las variables `femlab`, `literacy` y `school`.

In [1]:
# Utiliza esta celda para importar TODOS los módulos necesarios

In [2]:
# Utiliza esta celda para importar el set de datos según lo solicitado

### 2. Refactorizar gráficos matplotlib a seaborn (3 puntos). 

A partir de una serie de gráficos construidos con matplotlib, se solicita refactorizarlos utilizando seaborn. Además, se presenta la función que se utilizó para construirlos. Intente llegar al
resultado con mayor similitud. Comente los principales resultados de los gráficos.

**Histogramas**
```python
def binarize_histogram(dataframe, variable):
    tmp = dataframe.copy()
    tmp['binarize'] = np.where(tmp[variable] > np.mean(tmp[variable]), 1, 0)
    hist_1 = tmp[tmp['binarize'] == 1][variable].dropna()
    hist_0 = tmp[tmp['binarize'] == 0][variable].dropna()
    
    plt.subplot(1, 2, 1)
    plt.hist(hist_0, alpha=.6, color='lightgrey')
    plt.axvline(np.mean(hist_0))
    plt.title("{0} <= {1}".format(variable, round(np.mean(hist_0), 3)))
    
    plt.subplot(1, 2, 2)
    plt.hist(hist_1, alpha=.6, color='lightgrey')
    plt.axvline(np.mean(hist_1))
    plt.title("{0} >= {1}".format(variable, round(np.mean(hist_0), 3)))
    
binarize_histogram(df, 'adfert')
```



In [3]:
# Utiliza esta celda para definir la función binarize_histogram refactorizada
def binarize_histogram(dataframe, variable):
    # Se debe crear una copia del df original para no alterar el original
    tmp = dataframe.copy()
    
    # Se crea columna binarizada (1 si es mayor a la media, 0 sino)
    bin_col = f'binarize_{variable}'
    media_var = np.mean(tmp[variable])
    tmp[bin_col] = np.where(tmp[variable] > media_var, 1, 0)
    
    # Se crea grilla de seaborn
    g = sns.FacetGrid(
        tmp,                             # Set de datos
        col=bin_col,                     # Nombre de la variable categórica por la que se creará cada gráfico de la grilla (1 para cada categoría)
        col_wrap=2,                      # Cantidad de columnas en la grilla (2 categorías -> 2 columnas)
        sharex=False                     # Queremos que cada gráfico tenga su propia escala en el eje x
    ).map(
        sns.histplot,                    # Gráfico a aplicar a cada elemento de la grilla
        variable,                        # Variable a la que se aplica el gráfico
        color="grey"                     # Los argumentos para modificar el histograma se pasan como kwargs en map
    ) 
    
    # Se obtiene la media para cada clase (0 ó 1) de la columna binarizada
    line_position = tmp.groupby(bin_col)[variable].mean().values

    # Se recorre cada figura de la grilla y su índice
    for i, ax in enumerate(g.axes.flat):
        # Se agrega en cada figura la línea vertical de la media
        ax.axvline(x=line_position[i], color='dodgerblue')
        
        # Símbolo mayor si es clase 1, menor o igual sino
        simbolo = '>' if i else '<='
        
        # Se agrega título para cada figura
        ax.set_title(f"{variable} {simbolo} {round(media_var, 2)}")

    plt.tight_layout()

In [4]:
# Utiliza esta celda para graficar los histogramas solicitados utilizando la función refactorizada para tu variable 1

**Comentarios**

In [5]:
# Utiliza esta celda para graficar los histogramas solicitados utilizando la función refactorizada para tu variable 2

**Comentarios**

In [6]:
# Utiliza esta celda para graficar los histogramas solicitados utilizando la función refactorizada para tu variable 3

**Comentarios**

**Boxplots**
```python
def grouped_boxplot(dataframe, variable, group_by):
    tmp = dataframe
    stratify_by = tmp[group_by].unique()
    
    if len(stratify_by) / 2 > 3:
        fig, ax = plt.subplots(2, len(stratify_by),sharey=True)
    else:
        fig, ax = plt.subplots(1, len(stratify_by),sharey=True)
    
    for i, n in enumerate(stratify_by):
        ax[i].boxplot(tmp[tmp[group_by] == n][variable])
        ax[i].set_title(n)

grouped_boxplot(df, 'adfert', 'region')
```

In [7]:
# Utiliza esta celda para graficar los boxplots solicitados utilizando seaborn para tu variable 1

**Comentarios**

In [8]:
# Utiliza esta celda para graficar los boxplots solicitados utilizando seaborn para tu variable 2

**Comentarios**

In [9]:
# Utiliza esta celda para graficar los boxplots solicitados utilizando seaborn para tu variable 3

**Comentarios**

**Scatterplots**

```python
def grouped_scatterplot(dataframe, x, y, group_by):
    tmp = dataframe
    stratify_by = tmp[group_by].unique()

    if len(stratify_by) / 2 > 3:
        fig, ax = plt.subplots(2, len(stratify_by),sharey=True)
    else:
        fig, ax = plt.subplots(1, len(stratify_by),sharey=True)

    for i, n in enumerate(stratify_by):
        tmp_group_plt = tmp[tmp[group_by] == n]
        ax[i].plot(tmp_group_plt[x], tmp_group_plt[y], 'o')
        ax[i].set_title(n)
        
grouped_scatterplot(df, 'school', 'adfert', 'region')
```

In [10]:
# Utiliza esta celda para graficar los scatterplots solicitados utilizando seaborn para region y tu variable 1

**Comentarios**

In [11]:
# Utiliza esta celda para graficar los scatterplots solicitados utilizando seaborn para region y tu variable 2

**Comentarios**

In [12]:
# Utiliza esta celda para graficar los scatterplots solicitados utilizando seaborn para region y tu variable 3

**Comentarios**

### 3. Generar un heatmap entre todas las variables (2 puntos):
Con base en las variables de interés asignadas, comente cuáles son las principales correlaciones existentes, tomando como criterio de corte aquellas superior a .6

In [13]:
# Usa esta celda para generar df de correlaciones solicitado, con valor absoluto

In [14]:
# Usa esta celda para generar heatmap solicitado, con valores > 0.6

**Comentarios**

###  4. A partir de las principales correlaciones, separarlas en un nuevo objeto y calcular la matriz de correlaciones para todas las regiones (3 puntos):
- Comente brevemente las principales correlaciones a través de las regiones.

In [15]:
# Usa esta celda para generar df con las variables con principales correlaciones, de ser necesario

In [16]:
# Usa esta celda para generar la matriz de correlaciones para cada región

In [17]:
# Usa esta celda para mostrar las correlaciones de Africa

**Comentarios**

In [18]:
# Usa esta celda para mostrar las correlaciones de Americas

**Comentarios**

In [19]:
# Usa esta celda para mostrar las correlaciones de Asia

**Comentarios**

In [20]:
# Usa esta celda para mostrar las correlaciones de Europe

**Comentarios**

In [21]:
# Usa esta celda para mostrar las correlaciones de Oceanía

**Comentarios**